In [1]:


from GenZ import decode_moddeling, prefill_moddeling

import pandas as pd
from plotnine import *
import plotnine as p9
import pandas as pd
from GenZ.analyse_model import *


In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
D = get_configs('GPT-4').hidden_size
V = 50257
model_df, summary_table = decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 1 , bits='int8',
                            input_tokens = 1, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1, debug=True) 
display(model_df)
display(summary_table)
# print(f'Model Weights={summary_table['Model Weights (MB)'][0]*2**20/1e12} T')
# print(f'Attn Size:{(model_df.loc[0,'Input_w (MB)'] + model_df.loc[5,'Input_w (MB)'])*120*2**20/1e9} B')
# print(f'Each MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120*2**20/1e9/16} B')
# print(f'FWD pass MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')

/opt/homebrew/Caskroom/miniconda/base/envs/genz_wheel_test/lib/python3.11/site-packages/GenZ/LLM_inference/llm_decode.py:106: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 1819.125 GB, KV Cache:0.0025634765625 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,GEMM,"[1, 32256, 1, 10752]",1.999752,693.633024,0.010254,330.75,0.030762,330.791016
1,Logit,"[1, 84, 1, 1, 128]",0.996109,0.021504,0.010254,0.010254,0.00008,0.020588
2,Logit,"[1, 84, 1, 0, 128]",0.0,0.0,0.010254,0.0,0.0,0.010254
3,Attend,"[1, 84, 1, 1, 128]",0.996109,0.021504,0.00008,0.010254,0.010254,0.020588
4,Attend,"[1, 84, 1, 0, 128]",0.0,0.0,0.0,0.0,0.010254,0.010254
5,GEMM,"[1, 10752, 1, 10752]",1.999628,231.211008,0.010254,110.25,0.010254,110.270508
6,GEMM,"[1, 86016, 1, 10752]",1.999791,1849.688064,0.010254,882.0,0.082031,882.092285
7,GEMM,"[1, 602112, 0, 10752]",0.0,0.0,0.0,6174.0,0.0,6174.0
8,GEMM,"[1, 10752, 1, 86016]",1.999791,1849.688064,0.082031,882.0,0.010254,882.092285
9,GEMM,"[1, 10752, 0, 602112]",0.0,0.0,0.0,6174.0,0.0,6174.0


,MACs (MFLOP),Total Data (MB),Model Weights (MB),Unused Weights (MB),KV Cache (MB),AR data (MB),Pipe data (MB)
0,591905.685504,1.862823e+06,1862784.0,1580544.0,2.625,2.625,0.0


In [4]:
for model in ['gpt-4']:
    model_df, summary_table = prefill_moddeling(model = model, batch_size = 1,
                            input_tokens = 1024, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 8, pipeline_parallel = 16) 
    print(model)
    display(model_df)
    display(summary_table)


gpt-4


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,GEMM,"[1, 4160, 1024, 10752]",763.386128,91603.59936,21.0,85.3125,8.125,114.4375
1,Logit,"[1, 11, 1024, 1024, 128]",102.4,2952.790016,2.75,2.75,22.0,27.5
2,Attend,"[1, 11, 1024, 1024, 128]",102.4,2952.790016,22.0,2.75,2.75,27.5
3,GEMM,"[1, 10752, 1024, 1344]",551.384615,29595.009024,2.625,27.5625,21.0,51.1875
4,GEMM,"[1, 86016, 128, 10752]",126.30837,236760.072192,2.625,1764.0,21.0,1787.625
5,GEMM,"[1, 10752, 128, 86016]",126.30837,236760.072192,21.0,1764.0,2.625,1787.625


,MACs (MFLOP),Total Data (MB),Model Weights (MB),Unused Weights (MB),KV Cache (MB),AR data (MB),Pipe data (MB)
0,4.804995e+06,30367.0,29127.0,0,44.0,336.0,315.0


In [5]:
prefill_moddeling(model = 'gpt-4', batch_size = 1,
                                            input_tokens = 2000, output_tokens = 256, FLAT = True,
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False, time_breakdown=True) 

{'Latency': np.float64(112.47223831441586),
 'Throughput': np.float64(142.1932235057803),
 'Runtime_breakdown': [np.float64(81.8676871420636),
  np.float64(1.4578281092012133),
  29.146723063151043],
 'is_offload': False}

In [6]:
decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 4 ,
                                            input_tokens = 2000, output_tokens = 256, FLAT = True,
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False, time_breakdown=True) 

{'Latency': np.float64(19.022237012576436),
 'Throughput': np.float64(840.8428472783806),
 'Runtime_breakdown': [np.float64(10.191345214843752),
  np.float64(0.2995038874009076),
  8.582211446126303],
 'is_offload': False}